# Fantasy Premier League Moneyball

In [ ]:
# The plan:

# 1. Pull data from fantasy premier league site to create datasets for:
    # Club data - DONE
    # Player data - DONE
    # Fixture data - DONE
    # My team data - DONE
    # Previous season player data - NEED TO CREATE DF FROM CSV :
        #'C:\Users\andre\Documents\GitHub\murry_code\Fantasy Premier League\FPL_Players-2019-2020 season'
    
# 2. Create player score based on:
    # A - Points per million this season - DONE
    # B - Points per average player cost this season
    # C - Points per million last season (proportionate to games played by team so far this season, with a lower weighting)
    # D - Points per average player cost last season (as above, trialling ration of 125% this season 75% previous season)

    # E - Next gameweek multiplier (Includes Matches per gameweek multiplier)
    # F - Next 5 gameweek average multiplier
    
    # G - Chances of playing next round multiplier
    
    # Player score = (AVERAGE(A x 1.25,B x 1.25 ,(C x 0.75), (D x 0.75)) x AVERAGE(E, F)) x G  
    
# 3. Create Optimal team:    
    # Create best team of: 
        # Using 80-82 mil
        # 1 x GK
        # 3 x DEF
        # 4 x MID
        # 3 X FOR

    # Leaving enough for Subs: 
        # 18-20mil
        # 1 x GK
        # 2 x DEF
        # 1 X MID

# 4. Figure out a way to make transfer most efficiently to maximise my team score, 
    # Considering 1 free transfer, then 4 points per transfer after that       
    
    
# Additional ideas
    # Pull data from promoted teams from the EFL championship in previous year so there is data on newly promoted teams/players


### Creating variables for the api urls and logging in using personal details

In [130]:
#############
## imports ##
#############

import pandas as pd
from datetime import datetime as dt
import requests
import numpy as np

######################################################################################

###########################
## URLs an Login Details ##
###########################

# Code for my team picks
my_team_id = '3366741'

# Current Datetime for csv outputs
datetime = dt.now().strftime("%Y%m%d-%H%M%S")

## URLs

## login url to create a session using my login details ## 
login_url = 'https://users.premierleague.com/accounts/login/'

## api url for the main dataset on player stats 
main_url = 'https://fantasy.premierleague.com/api/bootstrap-static/'

## api for my team
team_url = 'https://fantasy.premierleague.com/api/my-team/3366741/'

## api for ???
entry_url = 'https://fantasy.premierleague.com/api/entry/3366741/'

## api for fixtures
fix_url = 'https://fantasy.premierleague.com/api/fixtures/'
fdr_url = 'https://fantasy.premierleague.com/api/fdr/'

session = requests.session()

## Create session with login details for urls which require login ##
payload = {
 'password': 'Football!159',
 'login': 'andrew_c_morris@hotmail.co.uk',
 'redirect_uri': 'https://fantasy.premierleague.com/a/login',
 'app': 'plfpl-web'
}
session.post(login_url, data=payload)


<Response [200]>

### Creating data frames and csv outputs

In [131]:
## CLUB DATAFRAME AND CSV EXTRACT ##
### Creating a dataframe for the club data and saving as csv ###

############################
#### dataframe creation ####
############################

## Converting the data from the url to json then pulling out the teams data to a separate 'club' data frame
main_req = requests.get(main_url)
main_json = main_req.json()
main_json.keys()
club_df = pd.DataFrame(main_json['teams'])

# print(club_df)

#####################
##### csv output ####
#####################

## Specifying the filename and path (Location to export the CSV to)
club_filename = 'FPL_Clubs-'
club_path = 'C:\\Python CSV Output\\'+club_filename+datetime+'.csv'

## Saved dataframe as csv
club_df.to_csv(club_path, index = False)

In [132]:
### PLAYER DATA EXTRACT ###
### Creating a dataframe for the player data and saving as csv ###

############################
#### Dataframe Creation ####
############################

full_player_df = pd.DataFrame(main_json['elements'])         # Player data
position_types_df = pd.DataFrame(main_json['element_types']) # Position data
team_df = pd.DataFrame(main_json['teams'])                   # Club data
events_df = pd.DataFrame(main_json['events'])                # Gameweek data

# print(team_df.head())

# Updating and creating some of the fields in full player dataframe: 
    # Pulling through position and team name instead of 'id', and changing data types
full_player_df['position'] = full_player_df.element_type.map(position_types_df.set_index('id').singular_name)
full_player_df['position_abv'] = full_player_df.element_type.map(position_types_df.set_index('id').singular_name_short)
full_player_df['team'] = full_player_df.team.map(team_df.set_index('id').name)
full_player_df['team_abv'] = full_player_df.team.map(team_df.set_index('name').short_name)

    # Converting selected_by_percent to float data type
full_player_df['selected_by_percent'] = full_player_df.selected_by_percent.astype(float)

    # Creating chance of playing next round multiplier
full_player_df['next_round_play_chance'] = full_player_df['chance_of_playing_next_round']/100.0
full_player_df['next_round_play_chance'] = full_player_df.next_round_play_chance.fillna(1.0)
full_player_df['this_round_play_chance'] = full_player_df['chance_of_playing_this_round']/100.0
full_player_df['this_round_play_chance'] = full_player_df.this_round_play_chance.fillna(1.0)

    # Creating custom point_per_mil
full_player_df['cost_in_mil'] = full_player_df['now_cost']/10
full_player_df['point_per_mil'] = round(full_player_df['total_points']/full_player_df['cost_in_mil'],2)

# Creating a slimline version of the elements table with relevant data points required for players
slim_player_df = full_player_df[['first_name',
                              'second_name',
                              'id',   
                              'position',
                              'position_abv',
                              'team',
                              'team_abv',
                              'selected_by_percent', 
                              'this_round_play_chance',   
                              'next_round_play_chance',      
                              'status',
                              'cost_in_mil',
                              'minutes',
                              'transfers_in',
                              'total_points',
                              'point_per_mil']]

# print(slim_player_df.head(10))


#####################
##### csv output ####
#####################

## Specifying the filename and path (Location to export the CSV to)
player_filename = 'FPL_Players-'
player_path = 'C:\\Python CSV Output\\'+player_filename+datetime+'.csv'

# ## Exporting dataframe as csv
slim_player_df.to_csv(player_path, index = False, encoding="utf8")

In [133]:
### FIXTURE DATA EXTRACT ###
### Creating a dataframe for the fixtures data and saving as csv ###

############################
#### dataframe creation ####
############################

## Getting fixture data from the api url
fix_req = requests.get(fix_url)

## Converting that data to json then dataframe
fix_json = fix_req.json()
fix_df = pd.DataFrame(fix_json)

# Adding club names and abbreviations as new columns
fix_df['away_team_full']  = fix_df.team_a.map(team_df.set_index('id').name)
fix_df['away_team_short'] = fix_df.team_a.map(team_df.set_index('id').short_name)
fix_df['home_team_full']  = fix_df.team_h.map(team_df.set_index('id').name)
fix_df['home_team_short'] = fix_df.team_h.map(team_df.set_index('id').short_name)

# Renaming 'event' to 'gameweek'
fix_df = fix_df.rename(columns={"event": "gameweek"})


## Slimming down the dataframe to only the columns we need
slim_fix_df = fix_df[['gameweek',
                      'kickoff_time',
                      'away_team_full',
                      'away_team_short',
                      'home_team_full',
                      'home_team_short',
                      'team_h_difficulty',
                      'team_a_difficulty']]


# print(slim_fix_df.head())


#####################
##### csv output ####
#####################

## Specifying the filename and path (Location to export the CSV to)
fixtures_filename = 'FPL_Fixtures-'
fixture_path = 'C:\\Python CSV Output\\'+fixtures_filename+datetime+'.csv'

## Exporting dataframe as csv
slim_fix_df.to_csv(fixture_path, index = False)

In [134]:
## MY TEAM PICKS DATAFRAME AND CSV EXTRACT ##
### Creating a dataframe for my team picks data and saving as csv ###

############################
#### dataframe creation ####
############################

## Getting my team data from the api url, using session as login details required
my_team_ses = (session.get('https://fantasy.premierleague.com/api/my-team/3366741'))

## Converting that data to json then dataframe
my_team_json = my_team_ses.json()
my_team_picks_df = pd.DataFrame(my_team_json['picks'])

# Pulling through player name data from 'full_player_df'
my_team_picks_df['first_name'] = my_team_picks_df.element.map(full_player_df.set_index('id').first_name)
my_team_picks_df['second_name'] = my_team_picks_df.element.map(full_player_df.set_index('id').second_name)
my_team_picks_df['potential_profit'] = my_team_picks_df.purchase_price - my_team_picks_df.purchase_price

# print(my_team_picks_df)


#####################
##### csv output ####
#####################

## Specifying the filename and path (Location to export the CSV to)
my_team_filename = 'FPL_My_Team-'
my_team_path = 'C:\\Python CSV Output\\'+my_team_filename+datetime+'.csv'

## Exporting dataframe as csv
my_team_picks_df.to_csv(my_team_path, index = False)

### Random Bits

In [ ]:
####### Show Datatypes Of Dataframe ##########
# dataTypeSeries = <insert dataframe name>.dtypes
# print('Data type of each column of Dataframe :')
# print(dataTypeSeries)
##############################################